In [199]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from operator import itemgetter
import seaborn as sns

In [200]:
def add_data_info():
    df1 = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-from-2017.csv', encoding= 'unicode_escape', low_memory=False)
    df2 = pd.read_csv('raw_data/charting-m-matches.csv', encoding= 'unicode_escape', low_memory=False)
    
    #get separated values from m-matches list of games - based on matchid
    df = pd.merge(df1,df2[['match_id','Date','Tournament','Round','Surface','Player 1','Player 2']], on=['match_id'], how='left')
    
    #add the condition of "Grand Slam" - 5 or 3 sets
    conditions = df['Tournament'].isin(["Wimbledon", "Australian Open", "US Open", "Roland Garros"])
    values = [1,0]

    # Create a new column  based on the conditions and values
    df['Grand Slam'] = pd.Series(pd.NA)
    df.loc[conditions, 'Grand Slam'] = values[0]
    df.loc[~conditions, 'Grand Slam'] = values[1]
    
    # save the original dataset with enriched data for future features
    df.to_csv('raw_data/charting-m-points-from-2017-enriched.csv', index=False)

In [201]:
def filter_player(df,opponent):
    
    # Read the dataset into a pandas DataFrame - from enriched file
    df = pd.read_csv(df, encoding= 'unicode_escape', low_memory=False)
    
    ## filtra pelo  oponente
    df = df[(df["Player 1"].str.contains(opponent)) | (df["Player 2"].str.contains(opponent))].copy()
    
    ##define se está sacando e se ganhou o ponto
    df['i_serve'] = np.where(((df['Svr'] == 1) & (df['Player 1'] == opponent )) | ((df['Svr'] == 2) & (df['Player 2'] == opponent)), 0, 1)
    df['i_win'] = np.where(((df['isSvrWinner'] == df['i_serve'])), 1, 0)
    df['is_second_service'] = np.where(((df['2nd'].isnull())), 0, 1)
    
    ## direção do saque
    # primeiro caractere do segundo saque
    df['dir_srv'] = df.loc[:, '2nd'].str[0]
    ## completa com o primeiro caractere do primeiro saque quando ele entrou
    df['dir_srv'].fillna(df.loc[:, '1st'].str[0], inplace=True)

    
    ##reseta o indice
    df.reset_index(inplace=True)
    
    ## salva arquivo e retorna dataset da funcao
    csv_file = "raw_data/" + opponent + "_points.csv"
    df.to_csv(csv_file, index=False)
    return df

In [202]:
meaning_char = pd.read_csv('https://raw.githubusercontent.com/eduardoscovino/tennis-analytics/master/code_map.csv',
                           encoding= 'unicode_escape', sep=';')

In [203]:
add_data_info()

In [204]:
df = filter_player('raw_data/charting-m-points-from-2017-enriched.csv','Novak Djokovic')

In [205]:
## serve direction encode 
df = df[df['dir_srv'].isin(['4','5','6'])]

## to numeric for rallyCount and serve direction
df['rallyCount']=pd.to_numeric(df['rallyCount'])
df['dir_srv']=pd.to_numeric(df['dir_srv'])

In [206]:
#Spliting Serve and Return
df['rally_desc'] = df.loc[:, '2nd']
df['rally_desc'].fillna(df.loc[:, '1st'], inplace=True)

##df['Serve'] = df.loc[:, 'rally_desc'].str[0:1]

df['rally_desc'] = df['rally_desc'].str[1:]

sc = np.array(meaning_char[~meaning_char['code'].isin(['f', 'b', 'r', 's', 'v', 'l', 'o', 'z', 'p', 'u', 'y', 'h', 'i', 'j', 'k','m', '1', '2', '3', '7', '8', '9'])]['code'])

#Spliting every shot
df['rally_desc'] = df['rally_desc'].apply(lambda x: ''.join([c for c in x if c not in sc]))
df['rally_desc'] = df['rally_desc'].str.replace('f',' f').str.replace('b',' b').str.replace('s',' s').str.replace('r',' r').str.replace('v',' v').str.replace('l',' l').str.replace('o',' o').str.replace('z',' z').str.replace('p',' p').str.replace('u',' u').str.replace('y',' y').str.replace('h',' h').str.replace('i',' i').str.replace('j',' j').str.replace('k',' k').str.replace('m',' m').str.replace('!','').str.replace('+','').str.replace(';','').str.replace('^','').str.replace('C','')
df['rally_desc'] = df['rally_desc'].map(lambda x: x.lstrip(' ').rstrip(' '))

df['rally_tratada'] = (df['rallyCount']/2).apply(np.floor).astype('int')

/var/folders/kk/0r0_hyfd673456ly0hptrbsc0000gn/T/ipykernel_72697/3403707337.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rally_desc'] = df['rally_desc'].str.replace('f',' f').str.replace('b',' b').str.replace('s',' s').str.replace('r',' r').str.replace('v',' v').str.replace('l',' l').str.replace('o',' o').str.replace('z',' z').str.replace('p',' p').str.replace('u',' u').str.replace('y',' y').str.replace('h',' h').str.replace('i',' i').str.replace('j',' j').str.replace('k',' k').str.replace('m',' m').str.replace('!','').str.replace('+','').str.replace(';','').str.replace('^','').str.replace('C','')


### Adição da coluna how_ended

In [207]:
# df['end']
df['how_ended'] = df.loc[:, '2nd'].str[-1]
df['how_ended'].fillna(df.loc[:, '1st'].str[-1], inplace=True)

In [208]:
df['isUnforced'] = df['isUnforced'].replace({True: 1, False: 0})
df['isForced'] = df['isForced'].replace({True: 1, False: 0})
df['serve_return'] = df['rally_desc'].map(lambda x: x.split(' ')).str[0]

### DATASET FINAL 

In [209]:
# Data set final sendo "i serve = 0" - receiving
df = df[df['i_serve'] == 0]
df = df[df['serve_return'] != '']
df = df[df['i_win'] == 1]

## Dataset finalizado ##

In [210]:
df.head()

,index,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,TBpt,Svr,Ret,Serving,1st,2nd,Notes,1stSV,2ndSV,1stIn,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,Date,Tournament,Round,Surface,Player 1,Player 2,Grand Slam,i_serve,i_win,is_second_service,dir_srv,rally_desc,rally_tratada,how_ended,serve_return
0,637,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,1,0,0,0,0,0-0,1 (1),1,0,NaN,1,2,ND,4n,4f18f3s2f1f3b3n@,NaN,0,0.0,0,1.0,False,False,False,0,1,False,2,0,6,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,0,1,1,4,f18 f3 s2 f1 f3 b3,3,@,f18
13,650,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,14,0,0,1,1,0-0,3 (1),1,0,NaN,1,2,ND,4r37b+3m2o2b2f3w@,NaN,NaN,0,NaN,1,NaN,False,False,False,0,1,False,2,0,6,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,0,1,0,4,r37 b3 m2 o2 b2 f3,3,@,r37
14,651,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,15,0,0,1,1,0-15,3 (2),1,0,NaN,1,2,ND,4n,5b28f3b2f3b3f2f2f3s1f1f1f3s3b3b1f2f3b3s3f1f1f1...,NaN,0,0.0,0,1.0,False,False,False,0,1,False,2,0,24,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,0,1,1,5,b28 f3 b2 f3 b3 f2 f2 f3 s1 f1 f1 f3 s3 b3 b1 ...,12,@,b28
33,670,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,34,0,0,3,3,30-0,7 (3),1,0,NaN,1,2,ND,4f17f+3s3z1w#,NaN,NaN,0,NaN,1,NaN,False,False,False,1,0,False,2,0,4,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,0,1,0,4,f17 f3 s3 z1,2,#,f17
54,691,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,55,0,0,5,5,15-0,11 (2),1,0,NaN,1,2,ND,4s37b+3b1*,NaN,NaN,0,NaN,1,NaN,False,False,True,0,0,False,2,0,4,20221120,Tour Finals,F,Hard,Novak Djokovic,Casper Ruud,0,0,1,0,4,s37 b3 b1,2,*,s37


In [243]:
df.iloc[

KeyError: 109

In [255]:
idx = df.groupby('match_id')['Pt'].transform(max) == df['Pt']
df['winner'] = df.apply(lambda row: row['Player 1'] if row['i_win'] == 1 else row['Player 2'], axis=1)
report = df[idx][['match_id','winner']]
report

,match_id,winner
109,20221120-M-Tour_Finals-F-Novak_Djokovic-Casper...,Novak Djokovic
266,20221119-M-Tour_Finals-SF-Taylor_Fritz-Novak_D...,Taylor Fritz
506,20221118-M-Tour_Finals-RR-Novak_Djokovic-Danii...,Novak Djokovic
621,20221116-M-Tour_Finals-RR-Novak_Djokovic-Andre...,Novak Djokovic
744,20221114-M-Tour_Finals-RR-Stefanos_Tsitsipas_-...,Stefanos Tsitsipas
...,...,...
18452,20180519-M-Rome_Masters-SF-Rafael_Nadal-Novak_...,Rafael Nadal
18633,20170607-M-Roland_Garros-QF-Novak_Djokovic-Dom...,Novak Djokovic
18750,20170521-M-Rome_Masters-F-Novak_Djokovic-Alexa...,Novak Djokovic
19120,20170119-M-Australian_Open-R64-Novak_Djokovic-...,Novak Djokovic


## Geração das features

In [211]:
features = ['Surface',
            'Round',
            'Grand Slam',
            'is_second_service',
            #'dir_srv',
            #'rallyCount',
            #'rally_tratada',
            'isUnforced',
            'isForced',
            "serve_return",
            "how_ended"
            ]
X = df[features]

y = df[['dir_srv','rallyCount']]

In [212]:
feat_categorical = ['Surface','Round','serve_return','how_ended']
categorical_cols = ['Surface','Round','serve_return','how_ended']

In [213]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2569 entries, 0 to 19319
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Surface            2569 non-null   object
 1   Round              2569 non-null   object
 2   Grand Slam         2569 non-null   int64 
 3   is_second_service  2569 non-null   int64 
 4   isUnforced         2569 non-null   int64 
 5   isForced           2569 non-null   int64 
 6   serve_return       2569 non-null   object
 7   how_ended          2569 non-null   object
dtypes: int64(4), object(4)
memory usage: 180.6+ KB


In [214]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2569 entries, 0 to 19319
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   dir_srv     2569 non-null   int64
 1   rallyCount  2569 non-null   int64
dtypes: int64(2)
memory usage: 60.2 KB


## separação - treino, teste e validação

In [215]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_encoded = pd.DataFrame(encoder.fit_transform(X[feat_categorical]))
X_encoded.columns = encoder.get_feature_names_out(feat_categorical)


In [216]:
X_encoded.reset_index(drop=True, inplace=True)  # Reset index to match X
X.reset_index(drop=True, inplace=True)  # Reset index to match X_encoded
X = pd.concat([X, X_encoded], axis=1)
X = X.drop(categorical_cols, axis=1)

In [217]:
# Train-Val Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



print(len(X_train), 'train examples')
#print(len(X_val), 'validation examples')
print(len(X_test), 'test examples')

1798 train examples
771 test examples


## Definição do modelo

In [218]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

y_int = y.astype(int)
xgb_model = MultiOutputRegressor(XGBRegressor(objective="reg:squarederror"))
xgb_model.fit(X_encoded, y_int)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...))

In [169]:
#y_int = y.astype(int)
#xgb_model_2 = MultiOutputRegressor(XGBRegressor(objective="reg:linear"))
#xgb_model_2.fit(X_encoded, y_int)

In [170]:
# Instanciate model
#from xgboost import XGBRegressor
#xgb_model_3 = XGBRegressor(objective="multi:softmax", num_class=3)  # Assuming 3 serve directions (4, 5, 6)
#xgb_model_3.fit(X_encoded, y)

## Resultado final / evaluate

In [219]:
y_pred = xgb_model.predict(X_encoded)

In [220]:
from sklearn.metrics import mean_squared_error
for i in range(2):  # Two outputs: dir_srv and rally_tratada
    y_true = y.iloc[:, i]
    y_pred_single = y_pred[:, i]
    mse = mean_squared_error(y_true, y_pred_single)
    rmse = mse**0.5
    print(f"Root Mean Squared Error (RMSE) for Output {i+1}: {rmse}")

Root Mean Squared Error (RMSE) for Output 1: 0.7373455242107232
Root Mean Squared Error (RMSE) for Output 2: 3.8153175431060524


In [221]:
# Make predictions for the example input X
predictions = xgb_model.predict(X_encoded)

# Print the predictions for each output
print("Predicted dir_srv:", predictions[:, 0].round().astype(int))
print("Predicted rally_tratada:", predictions[:, 1].round().astype(int))

Predicted dir_srv: [4 5 5 ... 5 6 5]
Predicted rally_tratada: [4 5 6 ... 6 5 9]


In [227]:
example_X = pd.DataFrame({
    "Surface": ["clay"],
    "Round": ["Final"],
    "Grand Slam": [1],
    "is_second_service": [1],
    #"dir_srv": [6],
    "rally_tratada": [0],
    "isUnforced": [0],
    "isForced": [1],
    "serve_return": ["b28"],
    "how_ended": ["@"],
    #"i_win": [1],
    #"i_serve": [1]
})


In [228]:
# Perform one-hot encoding for the example input X using the same encoder
example_X_encoded = pd.DataFrame(encoder.transform(example_X[categorical_cols]))
example_X_encoded.columns = encoder.get_feature_names_out(categorical_cols)

In [229]:
X_encoded.head()

,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard,Round_BR,Round_F,Round_QF,Round_R128,Round_R16,Round_R32,Round_R64,Round_RR,Round_SF,serve_return_b,serve_return_b1,serve_return_b17,serve_return_b18,serve_return_b19,serve_return_b2,serve_return_b27,serve_return_b28,serve_return_b29,serve_return_b3,serve_return_b37,serve_return_b38,...,serve_return_r27,serve_return_r28,serve_return_r29,serve_return_r3,serve_return_r37,serve_return_r38,serve_return_r39,serve_return_s,serve_return_s1,serve_return_s17,serve_return_s18,serve_return_s19,serve_return_s2,serve_return_s27,serve_return_s28,serve_return_s29,serve_return_s3,serve_return_s37,serve_return_s38,serve_return_s39,serve_return_y27,how_ended_#,how_ended_*,how_ended_@,how_ended_C
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [230]:
# Make predictions for the example input X
predictions = xgb_model.predict(example_X_encoded)

# Print the predictions for each output
# Round the predicted values to the nearest integer
predicted_dir_srv = predictions[:, 0].round().astype(int)
predicted_rally_tratada = predictions[:, 1].round().astype(int)

# Print the rounded predictions for each output
print("Predicted dir_srv:", predicted_dir_srv)
print("Predicted rally_tratada:", predicted_rally_tratada)

Predicted dir_srv: [5]
Predicted rally_tratada: [5]
